### 초기 경로 설정 (클래스 이용을 위함)

In [ ]:
%cd /project/segmentation/smcho1201/segmentation_project

In [ ]:
# files and system
import os
import sys
import time
import random
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
from IPython.display import clear_output
# working with images
import cv2
import imageio
import scipy.ndimage
# import skimage.transform

import torchvision.transforms as transforms

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F


import torchsummary
from tqdm import notebook

sys.path.insert(0, '..')

# losses
from utils.metrics import iou_pytorch_eval, IoULoss, IoUBCELoss, DiceBCELoss , dice_pytorch_eval
from utils.metrics import iou_pytorch_test, dice_pytorch_test, precision_pytorch_test, recall_pytorch_test, fbeta_pytorch_test, accuracy_pytorch_test

# dataset
from utils.dataset import myDataSet

## 랜덤성을 배제한 환경 고정

In [ ]:
random_seed= 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # select device for training, i.e. gpu or cpu
print(DEVICE)

In [ ]:
_size = 224, 224
resize = transforms.Resize(_size, interpolation=0)

test_transforms = transforms.Compose([
                           transforms.Resize(_size, interpolation=0),
                       ])

In [ ]:
def mk_model(model_name):
    if model_name == 'Unet':
        from models.unet import UNet
        model = UNet(channel_in=3, channel_out=1)
        model = model.to(DEVICE) # load model to DEVICE
        return model
    
    if model_name == 'UNet++':
        from models.nnUNet.nnunet import Nested_UNet as UNet_2p
        model = UNet_2p(1,3)
        model = model.to(DEVICE) # load model to DEVICE
        return model
    
    if model_name == 'ResUNet+++CRF':
        from models.ResUNet_plus3_CRF import CompNet as ResUNet_3p_CRF
        model = ResUNet_3p_CRF()
        model = model.to(DEVICE) # load model to DEVICE
        return model
        
    if model_name == 'DeepLab_V3+':
        from models.DeepLab_V3_p.model import DeepLab as DeepLab_V3_p
        model = DeepLab_V3_p(backbone = 'resnet', num_classes = 1)   # backbone : 선택 (resnet, xception, mobilenet)
        model = model.to(DEVICE) # load model to DEVICE
        return model
        
    if model_name == 'segresnet':
        from models.model_library.models.segnet import SegResNet
        model = SegResNet(num_classes = 1)
        model = model.to(DEVICE) # load model to DEVICE
        return model
        
    if model_name == 'FCN':
        from models.FCN.vision.vision.torchvision.models.segmentation.fcn import fcn_resnet101
        model = fcn_resnet101(num_classes = 1)
        model = model.to(DEVICE) # load model to DEVICE
        return model

# 

# 

# 

# DICE

In [ ]:
models = ['FCN', 'segresnet', 'DeepLab_V3+', 'Unet', 'UNet++', 'ResUNet+++CRF']
models = ['UNet++']

BATCH_SIZE = 8
criterion = DiceBCELoss()


data_names = {'ISIC-2017':'ISIC', 'wound':'wound', 'Kvasir-SEG':'Kvasir', 'CVC-ClinicDB':'CVC',
             'breast_cancer_benign':'benign', 'breast_cancer_malignant':'malignant'}

data_names = {'CVC-ClinicDB':'CVC'}

lst_test = []
for model_name in models:
    model = mk_model(model_name)

    
    for data_name in data_names:
        # model = mk_model()
        if 'breast_cancer' in data_name:
            name = data_name.split('breast_cancer_')[1]
            test_images = sorted(glob.glob(f'/data/segmentation/breast-cancer/testset_{name}/images/*'))
            test_labels = sorted(glob.glob(f'/data/segmentation/breast-cancer/testset_{name}/labels/*'))
            test_images = [img for img in test_images if img.find('jpg')!= -1] # super pixels 이미지 제외

        else:
            test_images = sorted(glob.glob(f'/data/segmentation/{data_name}/testset/images/*'))
            test_labels = sorted(glob.glob(f'/data/segmentation/{data_name}/testset/labels/*'))
            test_images = [img for img in test_images if img.find('jpg')!= -1] # super pixels 이미지 제외

        model.load_state_dict(torch.load(f'checkpoints/ckpt_{model_name}_{data_names[data_name]}.pth')['net'])

        # 데이터셋 클래스 적용
        custom_dataset_test = myDataSet(test_images, test_labels, transforms=test_transforms)
        dataloader_test = torch.utils.data.DataLoader(custom_dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

        model.eval()
        test_dice = 0
        #test_loss, test_iou, test_dice = 0, 0, 0
        
        
        for i, (imgs, masks) in enumerate(dataloader_test):
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)

            with torch.no_grad():
                ## Prediction
                prediction = model(imgs)

            loss = criterion(prediction, masks)

    #         test_loss += loss.item()
    #         test_iou += iou_pytorch_eval(prediction, masks)
            test_dice += dice_pytorch_eval(prediction, masks)                                      # test_loss/(i+1), test_iou/(i+1), 
            print("\r {:} {:} Test. Dice: {:.8f}". format(model_name, data_names[data_name], test_dice/(i+1)), end="")
        
        
        
        lst_test.append((test_dice/(i+1)).cpu().numpy())

arr = np.array(lst_test)
arr = arr.reshape(6,6)

df = pd.DataFrame(arr.T)
df.to_csv('/project/segmentation/smcho1201/test_DICE.csv', index=False)


        #df_epoch_metric = pd.DataFrame(lst_test, columns = ['test_losses', 'test_iou', 'test_dice'], index = [data_name])

        #df_epoch_metric.to_csv(f'/project/segmentation/smcho1201/training_{model_name}/test_{data_name}.csv', index_label = [''])

In [ ]:
arr.T # 열 : 모델  / 행 : 데이터

# 

# 

# 

# IOU

In [8]:
models = ['FCN', 'segresnet', 'DeepLab_V3+', 'Unet', 'UNet++', 'ResUNet+++CRF']

BATCH_SIZE = 8
criterion = DiceBCELoss()


data_names = {'ISIC-2017':'ISIC', 'wound':'wound', 'Kvasir-SEG':'Kvasir', 'CVC-ClinicDB':'CVC',
             'breast_cancer_benign':'benign', 'breast_cancer_malignant':'malignant'}

lst_test = []
for model_name in models:
    model = mk_model(model_name)

    
    for data_name in data_names:
        # model = mk_model()
        if 'breast_cancer' in data_name:
            name = data_name.split('breast_cancer_')[1]
            test_images = sorted(glob.glob(f'/data/segmentation/breast-cancer/testset_{name}/images/*'))
            test_labels = sorted(glob.glob(f'/data/segmentation/breast-cancer/testset_{name}/labels/*'))
            test_images = [img for img in test_images if img.find('jpg')!= -1] # super pixels 이미지 제외

        else:
            test_images = sorted(glob.glob(f'/data/segmentation/{data_name}/testset/images/*'))
            test_labels = sorted(glob.glob(f'/data/segmentation/{data_name}/testset/labels/*'))
            test_images = [img for img in test_images if img.find('jpg')!= -1] # super pixels 이미지 제외

        model.load_state_dict(torch.load(f'checkpoints/ckpt_{model_name}_{data_names[data_name]}.pth')['net'])

        # 데이터셋 클래스 적용
        custom_dataset_test = myDataSet(test_images, test_labels, transforms=test_transforms)
        dataloader_test = torch.utils.data.DataLoader(custom_dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

        model.eval()
        test_iou = 0
        #test_loss, test_iou, test_dice = 0, 0, 0
        
        
        for i, (imgs, masks) in enumerate(dataloader_test):
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)

            with torch.no_grad():
                ## Prediction
                prediction = model(imgs)

            loss = criterion(prediction, masks)

            #test_loss += loss.item()
            test_iou += iou_pytorch_eval(prediction, masks)
            #test_dice += dice_pytorch_eval(prediction, masks)                                      # test_loss/(i+1), test_iou/(i+1), 
            print("\r {:} {:} Test. Dice: {:.8f}". format(model_name, data_names[data_name], test_iou/(i+1)), end="")
        
        
        
        lst_test.append((test_iou/(i+1)).cpu().numpy())

arr = np.array(lst_test)
arr = arr.reshape(6,6)

df = pd.DataFrame(arr.T)
df.to_csv('/project/segmentation/smcho1201/test_IOU.csv', index=False)


        #df_epoch_metric = pd.DataFrame(lst_test, columns = ['test_losses', 'test_iou', 'test_dice'], index = [data_name])

        #df_epoch_metric.to_csv(f'/project/segmentation/smcho1201/training_{model_name}/test_{data_name}.csv', index_label = [''])

 ResUNet+++CRF malignant Test. Dice: 0.5044

In [9]:
arr.T # 열 : 모델  / 행 : 데이터

array([[0.7612214 , 0.7438964 , 0.7689675 , 0.7398602 , 0.74499077,
        0.7324739 ],
       [0.7265097 , 0.7232313 , 0.7252167 , 0.7343531 , 0.71189195,
        0.73689014],
       [0.8103446 , 0.7951529 , 0.7891718 , 0.702561  , 0.6598387 ,
        0.6378765 ],
       [0.81176937, 0.77313024, 0.8037007 , 0.6961664 , 0.6375786 ,
        0.5825466 ],
       [0.74060076, 0.72025084, 0.7569311 , 0.7138868 , 0.68189055,
        0.6156441 ],
       [0.65017605, 0.6195327 , 0.665076  , 0.5633    , 0.59186035,
        0.5043809 ]], dtype=float32)

# 

# 

# 

# LOSS

In [11]:
models = ['FCN', 'segresnet', 'DeepLab_V3+', 'Unet', 'UNet++', 'ResUNet+++CRF']

BATCH_SIZE = 8
criterion = DiceBCELoss()


data_names = {'ISIC-2017':'ISIC', 'wound':'wound', 'Kvasir-SEG':'Kvasir', 'CVC-ClinicDB':'CVC',
             'breast_cancer_benign':'benign', 'breast_cancer_malignant':'malignant'}

lst_test = []
for model_name in models:
    model = mk_model(model_name)

    
    for data_name in data_names:
        # model = mk_model()
        if 'breast_cancer' in data_name:
            name = data_name.split('breast_cancer_')[1]
            test_images = sorted(glob.glob(f'/data/segmentation/breast-cancer/testset_{name}/images/*'))
            test_labels = sorted(glob.glob(f'/data/segmentation/breast-cancer/testset_{name}/labels/*'))
            test_images = [img for img in test_images if img.find('jpg')!= -1] # super pixels 이미지 제외

        else:
            test_images = sorted(glob.glob(f'/data/segmentation/{data_name}/testset/images/*'))
            test_labels = sorted(glob.glob(f'/data/segmentation/{data_name}/testset/labels/*'))
            test_images = [img for img in test_images if img.find('jpg')!= -1] # super pixels 이미지 제외

        model.load_state_dict(torch.load(f'checkpoints/ckpt_{model_name}_{data_names[data_name]}.pth')['net'])

        # 데이터셋 클래스 적용
        custom_dataset_test = myDataSet(test_images, test_labels, transforms=test_transforms)
        dataloader_test = torch.utils.data.DataLoader(custom_dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

        model.eval()
        test_loss = 0
        #test_loss, test_iou, test_dice = 0, 0, 0
        
        
        for i, (imgs, masks) in enumerate(dataloader_test):
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)

            with torch.no_grad():
                ## Prediction
                prediction = model(imgs)

            loss = criterion(prediction, masks)

            test_loss += loss.item()
            #test_iou += iou_pytorch_eval(prediction, masks)
            #test_dice += dice_pytorch_eval(prediction, masks)                                      # test_loss/(i+1), test_iou/(i+1), 
            print("\r {:} {:} Test. Dice: {:.8f}". format(model_name, data_names[data_name], test_loss/(i+1)), end="")
        
        
        
        lst_test.append((test_loss/(i+1)))

arr = np.array(lst_test)
arr = arr.reshape(6,6)

df = pd.DataFrame(arr.T)
df.to_csv('/project/segmentation/smcho1201/test_LOSS.csv', index=False)


        #df_epoch_metric = pd.DataFrame(lst_test, columns = ['test_losses', 'test_iou', 'test_dice'], index = [data_name])

        #df_epoch_metric.to_csv(f'/project/segmentation/smcho1201/training_{model_name}/test_{data_name}.csv', index_label = [''])

 ResUNet+++CRF malignant Test. Dice: 0.8518

In [12]:
arr.T # 열 : 모델  / 행 : 데이터

array([[0.49590104, 0.65180785, 0.53428461, 0.65734848, 0.48943686,
        0.50026572],
       [0.16408304, 0.36291931, 0.17025495, 0.18043468, 0.32577077,
        0.18262952],
       [0.5354659 , 0.50663877, 0.37187739, 0.6045604 , 0.5741565 ,
        0.60684647],
       [0.19693058, 0.27398576, 0.23929104, 0.38589547, 0.52779048,
        0.50545075],
       [0.38081227, 0.46018622, 0.3781466 , 0.39464181, 0.79218178,
        0.51817868],
       [0.58388782, 0.5287721 , 0.53404205, 0.68724543, 0.74559362,
        0.85175658]])

# 

# 

# 

# 결과확인

In [19]:
DICE = pd.read_csv('/project/segmentation/smcho1201/test_DICE.csv')
IOU = pd.read_csv('/project/segmentation/smcho1201/test_IOU.csv')
LOSS = pd.read_csv('/project/segmentation/smcho1201/test_LOSS.csv')

In [20]:
DICE

,0,1,2,3,4,5
0,0.847799,0.832859,0.851966,0.829580,0.835013,0.819933
1,0.831074,0.847064,0.824289,0.830471,0.810033,0.837722
2,0.877945,0.867238,0.861689,0.789182,0.751535,0.747169
3,0.876755,0.842266,0.871246,0.762239,0.728981,0.689651
4,0.812883,0.801309,0.835011,0.777914,0.766945,0.704840
5,0.753114,0.741560,0.772088,0.668712,0.708943,0.645505


In [15]:
IOU

,0,1,2,3,4,5
0,0.761221,0.743896,0.768968,0.739860,0.744991,0.732474
1,0.726510,0.723231,0.725217,0.734353,0.711892,0.736890
2,0.810345,0.795153,0.789172,0.702561,0.659839,0.637876
3,0.811769,0.773130,0.803701,0.696166,0.637579,0.582547
4,0.740601,0.720251,0.756931,0.713887,0.681891,0.615644
5,0.650176,0.619533,0.665076,0.563300,0.591860,0.504381


In [16]:
LOSS

,0,1,2,3,4,5
0,0.495901,0.651808,0.534285,0.657348,0.489437,0.500266
1,0.164083,0.362919,0.170255,0.180435,0.325771,0.182630
2,0.535466,0.506639,0.371877,0.604560,0.574157,0.606846
3,0.196931,0.273986,0.239291,0.385895,0.527790,0.505451
4,0.380812,0.460186,0.378147,0.394642,0.792182,0.518179
5,0.583888,0.528772,0.534042,0.687245,0.745594,0.851757
